In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.metrics import (
    precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix
)
from openpyxl import Workbook, load_workbook
import os

# Function to calculate classification metrics
def calculate_classification_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    gini = 2 * auc - 1
    confusion = confusion_matrix(y_true, y_pred)

    metrics = {
        'Metric': ['Precision', 'Recall', 'F1-Score', 'AUC ROC Score', 'GINI'],
        'Value': [precision, recall, f1, auc, gini]
    }
    return metrics, confusion

# Function to save classification results to Excel
def save_classification_results_to_excel(directory, model_name, sheet_name, results, confusion):
    file_name = os.path.join(directory, f'{model_name}_classification_results.xlsx')
    try:
        workbook = load_workbook(file_name)
    except FileNotFoundError:
        workbook = Workbook()
        if 'Sheet' in workbook.sheetnames:
            del workbook['Sheet']

    if sheet_name not in workbook.sheetnames:
        workbook.create_sheet(title=sheet_name)
    
    sheet = workbook[sheet_name]
    sheet.append(['Metric', 'Value'])
    
    for metric, value in zip(results['Metric'], results['Value']):
        sheet.append([metric, value])
    
    sheet.append([])
    sheet.append(['Confusion Matrix'])
    confusion_df = pd.DataFrame(confusion, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])
    header = [''] + confusion_df.columns.tolist()
    sheet.append(header)
    for idx, row in confusion_df.iterrows():
        sheet.append([idx] + row.tolist())

    workbook.save(file_name)
    print(f'Classification results saved to {file_name}, sheet: {sheet_name}')

# Main function to run classification metrics
def run_classification_metrics():
    directory_selector = widgets.Text(
        description='Directory Path:',
        layout=widgets.Layout(width='500px'),
        style={'description_width': 'initial'}
    )
    dir_confirm_button = widgets.Button(description='Confirm Directory', button_style='success')
    
    model_name_widget = widgets.Text(
        description='Model Name:',
        layout=widgets.Layout(width='500px'),
        style={'description_width': 'initial'}
    )
    
    dataset_type_widget = widgets.Dropdown(
        options=[('Select One', 'select_one'), ('Train', 'train'), ('Test', 'test')],
        description='Dataset Type:',
        layout=widgets.Layout(width='300px'),
        style={'description_width': 'initial'}
    )
    
    file_selector_pred = widgets.FileUpload(description='Select y_pred File')
    threshold_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='Threshold:')
    run_button = widgets.Button(description='Run Metrics', button_style='info')
    output = widgets.Output()
    
    selected_directory = {'path': None}
    selected_files = {'train': None, 'test': None}

    def on_dir_confirm_clicked(b):
        with output:
            clear_output()
            selected_directory['path'] = directory_selector.value.strip()
            if os.path.isdir(selected_directory['path']):
                # Find relevant files automatically
                all_files = os.listdir(selected_directory['path'])
                train_files = [f for f in all_files if 'train' in f.lower() and f.endswith('.csv')]
                test_files = [f for f in all_files if 'test' in f.lower() and f.endswith('.csv')]

                if train_files:
                    selected_files['train'] = os.path.join(selected_directory['path'], train_files[0])
                if test_files:
                    selected_files['test'] = os.path.join(selected_directory['path'], test_files[0])
                
                if not selected_files['train'] or not selected_files['test']:
                    print("Could not find appropriate train/test CSV files in the folder.")
                    return
                
                display(widgets.HTML("<b>Directory confirmed. Please proceed with model setup.</b>"))
                display(widgets.VBox([
                    model_name_widget,
                    dataset_type_widget,
                    file_selector_pred,
                    threshold_slider,
                    run_button
                ]))
            else:
                print("Invalid directory. Please enter a valid path.")

    def on_run_button_clicked(b):
        with output:
            model_name = model_name_widget.value.strip()
            dataset_type = dataset_type_widget.value
            threshold = threshold_slider.value
            
            if not model_name:
                print("Model name is required.")
                return
            if dataset_type not in ['train', 'test']:
                print("Please select a dataset type.")
                return
            if not file_selector_pred.value:
                print("Please upload a prediction file.")
                return
            
            uploaded_file = next(iter(file_selector_pred.value.values()))['content']
            y_pred = pd.read_csv(pd.io.common.BytesIO(uploaded_file)).iloc[:, 0]
            
            y_actual_file = selected_files[dataset_type]
            y_actual = pd.read_csv(y_actual_file).iloc[:, 0]
            y_pred = (y_pred >= threshold).astype(int)
            
            results, confusion = calculate_classification_metrics(y_actual, y_pred)
            
            display(widgets.HTML(f'<h3>Model: <code>{model_name}</code> | Dataset: <code>{dataset_type}</code> | Threshold: <code>{threshold:.2f}</code></h3>'))
            display(pd.DataFrame(results))
            display(pd.DataFrame(confusion, columns=['Predicted 0', 'Predicted 1'], index=['Actual 0', 'Actual 1']))
            
            save_classification_results_to_excel(
                selected_directory['path'],
                model_name,
                os.path.basename(list(file_selector_pred.value.keys())[0]),
                results,
                confusion
            )

    dir_confirm_button.on_click(on_dir_confirm_clicked)
    run_button.on_click(on_run_button_clicked)
    
    display(widgets.VBox([
        widgets.HTML("<h2>Classification Metrics Evaluator</h2>"),
        widgets.HBox([directory_selector, dir_confirm_button]),
        output
    ]))
